In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from sklearn.manifold import TSNE
from pandas.plotting import parallel_coordinates
from sklearn import decomposition
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from math import  pi, exp,log, sqrt, fabs
import math
from scipy import stats
from datetime import date
from scipy.stats import norm
from sklearn.manifold import TSNE
import seaborn as sn
from sklearn.datasets import make_friedman1
from sklearn.decomposition import SparsePCA
from dask import dataframe as dd
import time
from numpy import abs

In [16]:
OP_df = pd.read_csv('Option2017_2_clean.csv')
OP_df = OP_df.dropna()
OP_df.head(3)

Unnamed: 0 currentDate  ExpDate  StrikePrice Ticker  Type  Last  \
0           0     2/17/17  7/21/17    34.799999   TBNK  Call  1.06   
1           2     2/17/17  3/17/17    22.500000    ARR  Call  0.08   
2           3     2/17/17  3/17/17    20.000000    ARR   Put  0.20   

   Implied volatility  Underlaying asset price  Expiration time  
0             0.13250                33.549999         0.421629  
1             0.13956                21.709999         0.076660  
2             0.32783                21.709999         0.076660

In [12]:
class callV():
    
    def __init__( self,last, S, K, r, T, tol = 0.000000000001):
        
        self.last = last
        
        self.S = S
        self.K = K
        
        self.r = r
        self.T = T
        
        self.sigma = 0
        self.tol = float(tol)
        
    def d11(self,sigma):
    
        x = ( log (self.S/self.K) + (( self.r + ((sigma**2)/2) )* self.T ))/( sigma * ( sqrt ( self.T )))
        return x 

    def d22(self,sigma):
    
        x = ( log (self.S/self.K) + (( self.r - ((sigma**2)/2) )* self.T ))/( sigma * ( sqrt ( self.T )))
        return x
    
    def call(self,sigma):

        return self.S  * norm.cdf( self.d11(sigma) ) - self.K * (exp(- (self.r*self.T) ) ) * norm.cdf( self.d22(sigma))

    def newcall(self, sigma = None):
        
        if sigma == None:
            sigma = self.sigma

        d1 = self.d11(sigma)
        d2 = self.d22(sigma)

        return self.call(sigma)
    
    def func(self, sigma = None):
        
        return self.newcall(sigma) - self.last
    

    def Muller(self, x, y, z):

        f1 = self.func(x)
        f2 = self.func(y)
        f3 = self.func(z)
        
        h0 = x - z
        h1 = y - z
        
        e0 = f1 - f3
        e1 = f2 - f3
        
        det = h0 * h1 * (h0 - h1)
        
        a = (e0 * h1 - h0 * e1) / det
        b =  ( ((h0**2) * e1)  - ((h1**2) * e0) ) / det
        c = f3

        if b < 0:
            
            w = z - 2 * c / (b - sqrt( (b**2) - (4*a*c) ))
        
        else:
            
            w = z - 2 * c / (b + sqrt( (b**2) - (4*a*c) ))

        return w

    def mullVol(self):
        
        i = 0
        
        a = self.tol
        c = 1
        b = (a + c) / 2

        while abs(self.func(b)) > self.tol:
            
            i = i + 1
            
            res= self.Muller(a,b,c)
            
            if self.func(a) * self.func(b) < 0:
                
                c = b
                
            else:
                
                a = b

            if res < a or res > c:
                
                b = (a+c) / 2
                
            else:
                
                b = res

        return b,i


In [13]:
class putV():
    
    def __init__( self,last, S, K, r, T,  tol = 0.000000000001):
        
        self.last = last
        
        self.S = S
        self.K = K
        
        self.r = r
        self.T = T
        
        self.sigma = 0
        self.tol = float(tol)
        
    def d11(self,sigma):
    
        x = ( log (self.S/self.K) + (( self.r + ((sigma**2)/2) )* self.T ))/( sigma * ( sqrt ( self.T )))
        return x 

    def d22(self,sigma):
    
        x = ( log (self.S/self.K) + (( self.r - ((sigma**2)/2) )* self.T ))/( sigma * ( sqrt ( self.T )))
        return x
    
    def put(self,sigma):

        return  -self.S  * norm.cdf(- self.d11(sigma) ) + self.K * (exp(- (self.r*self.T) ) ) * norm.cdf(- self.d22(sigma))

    def newcall(self, sigma = None):
        
        if sigma == None:
            sigma = self.sigma

        d1 = self.d11(sigma)
        d2 = self.d22(sigma)

        return self.put(sigma)
    
    def func(self, sigma = None):
        
        return self.newcall(sigma) - self.last
    
    def Muller(self, x, y, z):

        f1 = self.func(x)
        f2 = self.func(y)
        f3 = self.func(z)
        
        h0 = x - z
        h1 = y - z
        
        e0 = f1 - f3
        e1 = f2 - f3
        
        det = h0 * h1 * (h0 - h1)
        
        a = (e0 * h1 - h0 * e1) / det
        b =  ( ((h0**2) * e1)  - ((h1**2) * e0) ) / det
        c = f3

        if b < 0:
            
            w = z - 2 * c / (b - sqrt( (b**2) - (4*a*c) ))
        
        else:
            
            w = z - 2 * c / (b + sqrt( (b**2) - (4*a*c) ))

        return w

    def mullVol(self):
        
        i = 0
        
        a = self.tol
        c = 1
        b = (a + c) / 2

        while abs(self.func(b)) > self.tol:
            
            i = i + 1
            
            res= self.Muller(a,b,c)
            
            if self.func(a) * self.func(b) < 0:
                
                c = b
                
            else:
                
                a = b

            if res < a or res > c:
                
                b = (a+c) / 2
                
            else:
                
                b = res

        return b,i


0.16261914866754934 5


In [17]:
S = np.array( OP_df["Underlaying asset price"]).astype("float")
K = np.array( OP_df["StrikePrice"]).astype("float")
r = .03
T = np.array( OP_df["Expiration time"]).astype("float")
Op = np.array( OP_df["Last"]).astype("float")
Type = np.array( OP_df["Type"]).astype("str")
OP_df.head(3)

Unnamed: 0 currentDate  ExpDate  StrikePrice Ticker  Type  Last  \
0           0     2/17/17  7/21/17    34.799999   TBNK  Call  1.06   
1           2     2/17/17  3/17/17    22.500000    ARR  Call  0.08   
2           3     2/17/17  3/17/17    20.000000    ARR   Put  0.20   

   Implied volatility  Underlaying asset price  Expiration time  
0             0.13250                33.549999         0.421629  
1             0.13956                21.709999         0.076660  
2             0.32783                21.709999         0.076660

In [18]:
i=1
m,it = callV( Op[i] , S[i] , K[i] , r , T[i] ).mullVol()
print("Implied Volatility",m)
print("Iteration",it)

Implied Volatility 0.13273864438271182
Iteration 7


In [ ]:
imp = []
iter = []
for i in range( 0, len(OP_df) ):
    if(Type[i] == "Call"):
        try:
            
            m,it = (callV( Op[i] , S[i] , K[i] , r , T[i],.31 ).mullVol()) 
            imp.append(m)
            iter.append(it)
        except:
            imp.append(.31)
            iter.append(100)
    else:
        try:
            
            m,it = (putV( Op[i] , S[i] , K[i] , r , T[i],.31 ).mullVol()) 
            imp.append(m)
            iter.append(it)
        except:
            imp.append(.31)
            iter.append(100)

C:\Users\chsha\AppData\Local\Temp/ipykernel_14992/3612688761.py:69: RuntimeWarning: divide by zero encountered in double_scalars
  w = z - 2 * c / (b + sqrt( (b**2) - (4*a*c) ))
C:\Users\chsha\AppData\Local\Temp/ipykernel_14992/3612688761.py:59: RuntimeWarning: invalid value encountered in double_scalars
  a = (e0 * h1 - h0 * e1) / det
C:\Users\chsha\AppData\Local\Temp/ipykernel_14992/3612688761.py:60: RuntimeWarning: invalid value encountered in double_scalars
  b =  ( ((h0**2) * e1)  - ((h1**2) * e0) ) / det


In [11]:
OP_df['imp'] = pd.Series(imp)
OP_df['iter'] = pd.Series(iter)
OP_df.head(3)

Unnamed: 0 currentDate  ExpDate  StrikePrice Ticker  Type  Last  \
0           0     2/17/17  7/21/17    34.799999   TBNK  Call  1.06   
1           2     2/17/17  3/17/17    22.500000    ARR  Call  0.08   
2           3     2/17/17  3/17/17    20.000000    ARR   Put  0.20   

   Implied volatility  Underlaying asset price  Expiration time       imp  \
0             0.13250                33.549999         0.421629  0.162619   
1             0.13956                21.709999         0.076660  0.132739   
2             0.32783                21.709999         0.076660  0.341223   

   iter  
0     5  
1     7  
2     5

In [12]:
OP_df["diff"] = OP_df['Implied volatility'] - OP_df['imp']
OP_df['mse'] = OP_df['diff']**2
mse = OP_df['mse'].sum()/len(OP_df)
iteration = OP_df['iter'].sum()/len(OP_df)

In [13]:
n = 1 / ( ( 1 + mse ) * log( 1 + iteration ) )
n
n = 1 / ( ( 1 + mse ) * log( 1 + iteration ) )
print("MSE :",mse)
print("Average Iteration : ",iteration)
print("Average Efficiency",n)

MSE : 0.030415703560924896
Average Iteration :  26.13075268817204
Average Efficiency 0.29402597965727145


In [14]:
list = []
list.append(mse)
list.append(iteration)
list.append(n)


In [15]:
df = pd.read_csv('result.csv')

In [16]:
df["Muller-Bisection"] = list
df.head(3)

bisection     Brent  Newton-Raphson  New-Newton  Muller-Bisection  \
0   0.028464  0.029913        0.053913    0.029913          0.030416   
1  43.423779  9.502734        7.363687    9.617389         26.130753   
2   0.256294  0.412885        0.446747    0.410988          0.294026   

   New-Halley  
0    0.025485  
1    6.207127  
2    0.493729

In [17]:
df.to_csv('result.csv', index=False)